In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Our first task is to simulate the flow of heat in a solid.

The heat flow equation is:

$$ \frac{\partial u}{\partial t} = \frac{K}{C\rho}\nabla^2 u $$

$$ = \frac{K}{C\rho} \Big(\frac{\partial^2 u}{\partial x^2}
+ \frac{\partial^2 u}{\partial y^2}
+ \frac{\partial^2 u}{\partial z^2}\Big) $$

Where $u$ is the thermal energy density, $K$ is the thermal conductivity, $C$ is the specific heat, and $\rho$ is the density. For brevity, I will define $A = \frac{K}{C\rho}$.

We solve this using a timestep algorithm.

At a given point $u_i(t_{i})$, the spatial derivatives can be expressed in discrete form:
$$ \frac{\partial^2 u}{\partial x^2} = \frac{u_{i+1} - 2u_i + u_{i-1}}{\Delta x^2} $$

And the time derivative can be expressed as:
$$ \frac{\partial u}{\partial t} = \frac{u(t_{j+1}) - u(t_j)}{\Delta t} $$

Given a particular temperature distribution, we may find the next iteration using the following time step equation:

$$ u_{i,j,k}(t_{j+1}) = u_{i,j,k}(t_j)
 + A\Big(
         \frac{u_{i+1,j,k} - 2u_{i,j,k} + u_{i-1,j,k}}{\Delta x^2}
         + \frac{u_{i,j+1,k} - 2u_{i,j,k} + u_{i,j-1,k}}{\Delta y^2}
         + \frac{u_{i,j,k+1} - 2u_{i,j,k} + u_{i,j,k-1}}{\Delta z^2}
    \Big) \Delta t $$

If we take the same spatial step size for each dimension, we may simplify the expression in parentheses:
$$ \Big(u_{i+1,j,k} + u_{i,j+1,k} + u_{i,j,k+1} + u_{i-1,j,k} + u_{i,j-1,k} + u_{i,j,k-1} - 6u_{i,j,k}\Big)
\cdot\frac{1}{\Delta x^2}
$$
$$
= \frac{1}{\Delta x^2} \bigg(\sum_{adjacent} u(t_j) - 6u_{i,j,k}\bigg)
$$

Thus, we have:
$$ u_{i,j,k}(t_{j+1}) = u_{i,j,k}(t_j)
+ A\frac{\Delta t}{\Delta x^2} \bigg(\sum_{adjacent} u(t_j) - 6u_{i,j,k}\bigg)
$$

To handle an insulated boundary, we introduce fictional points such that the derivative will always be zero.

As a simplified case, we assume that the non-insulated boundaries are in perfect thermal contact with a heat sink at a constant temperature.


### First step: Boundary Conditions ###

First I write a function that updates the fictional points. For the insulated boundaries, the fictional points are just outside the boundary and have the same temperature as the boundary.

For non-insulated boundaries, the points are held at a fixed temperature.

In [ ]:
def updateBoundary(T, T_ext):
    """
    Updates the boundary points for the thermal conductor.
    The bottom surface of the conductor (where the first index is zero)
    is assumed to have perfect thermal contact with the surroundings,
    and will be held at temperature T_ext.
    
    The NumPy array is modified in-place.
    
    Arguments:
    ---------
    - T: a 3d array of temperatures within the solid. The shape of the
         array must be (h+1, w+2, d+2) to include the fictional points
         on 5 of the 6 surfaces.
         
    - T_ext: Temperature of the external environment
    
    Returns:
    ------
    Nothing
    
    """
    # Thermal conductive surface
    T[0,:,:] = T_ext
    
    # Insulated surfaces
    T[-1,:,:] = T[-2,:,:]
    T[:,0,:] = T[:,1,:]
    T[:,-1,:] = T[:,-2,:]
    T[:,:,0] = T[:,:,1]
    T[:,:,-1] = T[:,:,-2]
    

In [ ]:
def thermalSimulate(T, t, dx=0.1, A=1, T_ext=0):
    """
    Simulates the diffusion of heat through a solid with an initial
    temperature distribution given by T. One boundary of the solid
    (where the first index is zero) is assumed to be in perfect
    thermal contact with the surroundings. The other boundaries
    are insulated.
    
    Arguments:
    ---------
    - T: a 3D NumPy array containing the initial temperature distribution.
         
    - t: an array containing the simulation times; should have uniform separation
    
    - dx: the separation between adjacent points in the solid
    
    - A: the ratio K/(C*rho) where K is the conductivity, C is the specific
         heat capacity, and rho is the density
    
    - T_ext: the temperature of the external heat sink
    
    Returns:
    -------
    - simulation: a 4D NumPy array containing the results of the
         simulation, with the first three indices corresponding to
         the spatial coordinates, and the fourth index corresponding
         to the time
    
    """
    
    # Assume uniform times.
    dt = t[1]-t[0]
    
    
    # Modify the array to include fictional points
    currentT = np.vstack((T, T[[-1],:,:]))
    currentT = np.hstack((currentT[:,[0],:], currentT, currentT[:,[-1],:]))
    currentT = np.dstack((currentT[:,:,[0]], currentT, currentT[:,:,[-1]]))
    updateBoundary(currentT, T_ext)
    
    # Make an empty array with an extra axis for time
    simulation = np.zeros(T.shape + (len(t),))
    
    
    # First entry
    simulation[:,:,:,0] = T
    
    # Iterate!
    for i in range(1,len(t)):
        
        # Calculate the stuff as described in the project notebook
        T_sum = np.roll(currentT, -1, axis=0) + np.roll(currentT, 1, axis=0) \
              + np.roll(currentT, -1, axis=1) + np.roll(currentT, 1, axis=1) \
              + np.roll(currentT, -1, axis=2) + np.roll(currentT, 1, axis=2)
        deltaT = A*(T_sum - 6*currentT)/(dx**2) * dt
        
        # Update the temperature
        currentT = currentT + deltaT
        
        # Update the boundary, since we probably messed it up
        updateBoundary(currentT, T_ext)
        
        # Add to the simulation
        simulation[:,:,:,i] = currentT[:-1,1:-1,1:-1]
    
    return simulation

In [ ]:
q = np.random.rand(5)
q

In [ ]:
q[:-1]

In [ ]:
def thermalSimulate2(T, t, dx=0.1, K=0.01, CR=1, T_ext=0):
    """
    Simulates the diffusion of heat through a solid with an initial
    temperature distribution given by T. One boundary of the solid
    (where the first index is zero) is assumed to be in perfect
    thermal contact with the surroundings. The other boundaries
    are insulated.
    
    Arguments:
    ---------
    - T: a 3D NumPy array containing the initial temperature distribution.
         If the actual dimensions of the solid are i, j, and k, then the
         array should have dimensions i+1, j+2, and k+2.
         
    - t: an array containing the simulation times
    
    - dx: the separation between adjacent points in the solid
    
    - K: the thermal conductivity
    
    - CR: the specific heat capacity times the density, or equivalently,
        the heat capacity per unit volume.
    
    - T_ext: the temperature of the external heat sink
    
    Returns:
    -------
    - simulation: a 4D NumPy array containing the results of the
         simulation, with the first three indices corresponding to
         the spatial coordinates, and the fourth index corresponding
         to the time
    
    """
    # Make an empty array with an extra axis for time
    simulation = np.zeros(T.shape + (len(t),))
    
    # Assume uniform times.
    dt = t[1]-t[0]
    
    # Prefactor
    A = K/CR
    
    
    # First entry
    currentT = T.copy()
    updateBoundary(currentT, T_ext)
    simulation[:,:,:,0] = currentT
    
    # Iterate!
    for i in range(1,len(t)):
        
        # Calculate the stuff as described in the project notebook
        T_sum = np.roll(currentT, -1, axis=0) + np.roll(currentT, 1, axis=0) \
              + np.roll(currentT, -1, axis=1) + np.roll(currentT, 1, axis=1) \
              + np.roll(currentT, -1, axis=2) + np.roll(currentT, 1, axis=2)
        deltaT = A*(T_sum - 6*currentT)/(dx**2) * dt
        
        # Update the temperature
        currentT = currentT + deltaT
        
        # Update the boundary, since we probably messed it up
        updateBoundary(currentT, T_ext)
        
        # Add to the simulation
        simulation[:,:,:,i] = currentT
    
    return simulation

### Step 3: Testing the Heat Simulation ###

We carry out a simple test: if the solid begins at temperature different from the heat bath, will the temperature equalize?

In [ ]:
# Initial temperature 0K
T = np.zeros((11, 12, 12))

# Time span of 100 seconds
t=np.linspace(0, 100, 1000)

# Simulate with an external temperature of 10 K
sim = thermalSimulate(T, t, T_ext=10)


In [ ]:
# Display a cross section from the middle of the solid
# observed at the end of the simulation
plt.imshow(sim[:,5,:,-1])
plt.figure()
# Plot the temperature from the insulated to the
# non-insulated end
plt.plot(sim[:,5,5,-1])
plt.ylim(0,10)
plt.xlim(0,10)
plt.figure()

Now run the simulator using the values for a real substance. I'll use a cube of tin measuring 1 mm across. Since this is just a qualititive test, I'll grab the properties of tin from a quick Google search.

Specific heat capacity: 0.2177 J/(g$\cdot^{\circ}$C) [\[ref\]](https://www.engineersedge.com/materials/specific_heat_capacity_of_metals_13259.htm) \
Density: 5.769 g/cm$^3$  [\[ref\]](https://en.wikipedia.org/wiki/Tin) \
Thermal conductivity: 66.8 W/(m$\cdot$K)  [\[ref\]](https://en.wikipedia.org/wiki/Tin)

This gives CR = 1.256 J/(cm$^3 \cdot ^{\circ}$C).

In [ ]:
# Initial temperature 4K
T = np.ones((11, 11, 11))*4

# Parameters
K = 66.8 # Thermal conductivity
CR = 1.256 # Heat capacity times density
A = K/CR
dx = 1e-4 # Cube side length


# Pick a more stable timestep
dt = 0.1*dx**2/A

# Time span of 100 seconds
t=np.arange(0, dt*1000, dt)

# Simulate with an external temperature of 10 K
# sim = thermalSimulate(T, t, T_ext=10, CR=1.256, K=66.8, dx=0.0001)

sim = thermalSimulate(T, t, T_ext=10, A=A, dx=dx)

In [ ]:
# Display a cross section from the middle of the solid
# observed at the end of the simulation
plt.imshow(sim[:,5,:,-1])
plt.figure()
# Plot the temperature from the insulated to the
# non-insulated end
plt.plot(sim[:,5,5,-1])
plt.ylim(0,10.1)
plt.xlim(0,10)
plt.figure()

Now run the simulator with a single photon strike at the top center.

In [ ]:
# Initial temperature 0K
T = np.ones((7, 11, 11))*4
T[-1,5,5] = 44

# Time span of 10 seconds
t=np.arange(0, 10, 0.1)

# Simulate with an external temperature of 10 K
sim = thermalSimulate(T, t, T_ext=4)

In [ ]:
plt.imshow(sim[:,6,:,-1], vmin=4, vmax=4.05)
# plt.imshow(T[:,6,:])
plt.figure()

In [ ]:
t_signature = sim[1,6,6,:]
plt.plot(t,t_signature)

In [ ]:
def getSignature(p_loc, p_energy, dt=0.1, tmax=100, T_ext=4, sensor_loc=(1, 6, 6)):
    # Initial temperature
    T = np.ones((12, 13, 13))*T_ext
    T[p_loc[0], p_loc[1], p_loc[2]] = T_ext + p_energy

    t=np.arange(0, tmax, dt)

    sim = thermalSimulate(T, t, T_ext=T_ext)
    
    signature = sim[sensor_loc[0],sensor_loc[1],sensor_loc[2],:]
    
    return t, signature

In [ ]:

t, sig = getSignature((10, 1, 5), 100, tmax=300)
plt.plot(t, sig)